# Atender a diferentes partes de la entrada con autoatención

Ahora  abordaremos  el  funcionamiento  interno  del  mecanismo  de  autoatención  y  aprenderemos  a  codificarlo  desde  cero.  La  autoatención  es  la  piedra  angular  de  todo  LLM  basado  en  la  arquitectura  del  transformador. 

- En  la  autoatención,  el  "yo"  se  refiere  a  la  capacidad  del  mecanismo  para  calcular  ponderaciones  de  atención  relacionando  diferentes  posiciones  dentro  de  una  sola  secuencia  de  entrada.  Evalúa  y  aprende  las  relaciones  y  dependencias  entre  diversas  partes  de  la  propia  entrada,  como  las  palabras  en  una  oración  o  los  píxeles  en  una  imagen.  Esto  contrasta  con  los  mecanismos  de  atención  tradicionales,  donde  la  atención  se  centra  en  las  relaciones  entre  elementos  de  dos  secuencias  diferentes,  como  en  los  modelos  secuencia  a  secuencia,  donde  la  atención  podría  estar  entre  una  secuencia  de  entrada  y  una  secuencia  de  salida,

### Un mecanismo simple de autoatención sin pesos entrenables

El  objetivo  de  esta  sección  es  ilustrar  algunos  conceptos  clave  de  la  autoatención  antes  de  añadir  pesos  entrenables

![Texto alternativo](./imgs/3.7.png)


El objetivo de la **autoatención** es calcular, para cada elemento de entrada de una secuencia, un **vector de contexto** que combine información de todos los demás elementos de esa secuencia.  

- La figura muestra una secuencia de entrada \(x\), formada por \(T\) elementos, representados como $x^{(1)}, x^{(2)}, \dots, x^{(T)}$.  
- Esta secuencia normalmente corresponde a texto (por ejemplo, una oración) que ya ha sido convertido en **vectores de incrustación** (embeddings de tokens).  
  - Ejemplo: si el texto de entrada es *“Tu viaje comienza con un paso”*, cada token (“Tu”, “viaje”, “comienza”, …) se convierte en un vector de incrustación de dimensión \(d\).  
- En la figura, los vectores de entrada se muestran como incrustaciones tridimensionales para simplificar la visualización.

En la autoatención, lo que buscamos es calcular, para cada $x^{(i)}$, un **vector de contexto $z^{(i)}$**.  
- Este vector de contexto es un embedding enriquecido, ya que no solo contiene información sobre el token $x^{(i)}$ en sí, sino también sobre todos los demás tokens de la secuencia $x^{(1)}, \dots, x^{(T)}$.  
- Para lograrlo, se usan los **pesos de atención**, que indican la importancia relativa de cada elemento de entrada en el cálculo del contexto de un token concreto.  

Por ejemplo:  
- Si nos fijamos en el segundo token $x^{(2)} =$ “viaje”, el vector de contexto correspondiente es $z^{(2)}$.  
- Este $z^{(2)}$ combina información tanto de “viaje” como del resto de tokens (“Tu”, “comienza”, “con”, …).  
- Así, $z^{(2)}$ es una representación más rica y útil que la incrustación inicial de “viaje” aislada.  



In [5]:
import torch

#Considerar  la  siguiente  oración  de  entrada,  que  ya  ha  sido  incorporada  en  vectores  tridimensionales 
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     
    [0.55, 0.87, 0.66], # journey  (x^2)
    [0.57, 0.85, 0.64], # starts   (x^3)
    [0.22, 0.58, 0.33], # with     
    [0.77, 0.25, 0.10], # one      
    [0.05, 0.80, 0.55]] # step     
)

El  primer  paso  para  implementar  la  autoatención  es  calcular  los  valores  intermedios  ω,  denominados  puntuaciones  de  atención.

![Texto alternativo](./imgs/3.8.png)

1. **Consulta (query)**  
   - El segundo token de entrada, $x^{(2)}$, se considera la **consulta**.  
   - La consulta se compara con todos los demás tokens de la secuencia para determinar su importancia relativa.

2. **Cálculo de puntuaciones de atención**  
   - Se calculan como un **producto escalar** entre la consulta y cada token de la entrada:  
     $
     \text{score}(x^{(2)}, x^{(j)}) = x_{\text{consulta}} \cdot x^{(j)}
     $
   - Aquí, $x_{\text{consulta}} = x^{(2)}$.

3. **Normalización y truncado**  
   - Las cifras pueden truncarse a un dígito para simplificar la visualización.  
   - Después de calcular los productos escalares, se aplica **softmax** para obtener los **pesos de atención**.

4. **Vector de contexto**  
   - Finalmente, el vector de contexto $z^{(2)}$ se obtiene como la **combinación ponderada de todos los tokens de entrada**, usando los pesos de atención:
     $$
     z^{(2)} = \sum_{j=1}^{T} \text{peso\_atención}_j \cdot x^{(j)}
     $$


In [6]:
query = inputs[1]
att_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    att_scores_2[i] = torch.dot(query, x_i)
print(att_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


Normalizar los pesos de atención.

![Texto alternativo](./imgs/3.9.png)

El objetivo es obtener pesos de atención que sumen 1. Esta normalización es útil para la interpretación y para mantener la estabilidad del entrenamiento en un LLM.


In [7]:
att_weights_2_tmp = att_scores_2 / att_scores_2.sum()
print("Pesos de atencion: ", att_weights_2_tmp)
print("Suma: ", att_weights_2_tmp.sum())

Pesos de atencion:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Suma:  tensor(1.0000)


In [8]:
#Más recomendable utilizar la función softmax
def softmax(x):
    return torch.exp(x)/torch.exp(x).sum(dim=0)
att_weights_2_naive = softmax(att_scores_2)
print("Pesos de atencion: ", att_weights_2_naive)
print("Suma: ", att_weights_2_naive.sum())

#Garantiza que los pesos de atención siempre sean positivos

Pesos de atencion:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Suma:  tensor(1.)


In [9]:
#Utilizar softmax de torch ya que se tiene en cuenta problemas de inestabilidad, desbordamiento y subdesbordamiento
att_weights_2 = torch.softmax(att_scores_2, dim=0)
print("Pesos de atencion: ", att_weights_2)
print("Suma: ", att_weights_2.sum())

Pesos de atencion:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Suma:  tensor(1.)


Ahora que ya hemos calculado los pesos de atención normalizados, podemos realizar el paso final para obtener el vector de contexto 

## Cómo se hace

1. **Multiplicar cada embedding por su peso de atención**  
   - Cada token de entrada $x^{(j)}$ se multiplica por su peso de atención $\alpha_j$:
     $$
     x^{(j)} \times \alpha_j
     $$
   - Esto pondera cada token según su importancia para la consulta.

2. **Sumar los vectores ponderados**  
   - Después de multiplicar, se suman todos los vectores para formar el **vector de contexto final $z^{(i)}$**:
     $$
     z^{(i)} = \sum_{j=1}^{T} \alpha_j \, x^{(j)}
     $$
   - Aquí, $T$ es el número total de tokens en la secuencia.

- $z^{(i)}$ es una **representación enriquecida** de $x^{(i)}$, incorporando información de todos los demás tokens de la secuencia, ponderada por su relevancia.  
- Cada token “mira” a todos los demás y recoge la información más relevante para entender su contexto.

![Texto alternativo](./imgs/3.10.png)

In [10]:
query = inputs[1] #2nd input es la query
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += att_weights_2[i]*x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


## Cálculo de pesos de atención para todos los tokens de entrada

Se han calculamos  los  pesos  de  atención  y  el  vector  de  contexto  para  la  entrada  2, ahora   ampliar este  cálculo  para  calcular  los  pesos  de  atención  y  los  vectores  de  contexto  para  todas  las  entradas

![Texto alternativo](./imgs/3.11.png)

En la imagen se muestra de forma resaltada los pesos de atención respecto al segundo elemento.

Para lograr estos solo hay que seguir los mismos pasos anteriormente descrito, pero de forma generalizada.

1 - Calcular puntajes de atención 

2 - Calcular pesos de atención --> Normalización 

3 - Calcular vectores de contexto

In [ ]:
#1. Calcular los puntuajes de atencion
attn_scores = torch.empty(6, 6)
for i, i_x in enumerate(inputs):
    for j, j_y in enumerate(inputs):
        attn_scores[i, j] = torch.dot(i_x, j_y)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


Cada  elemento  del  tensor  anterior  representa  una  puntuación  de  atención  entre  cada  par  de entradas.

Debido a que los bucles for son lentos, se pueden obtener los mismos resultados con multiplicación de matrices.

In [13]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [24]:
#2. Calcular los pesos de atencion
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

#Al establecer dim=1, se indica a softmax que la normalización sea a lo largo de las columnas, 
#ya que el tensor es 2D de la forma [filas, columnas].
#Por lo tanto la suma de cada columnas debe ser 1

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [22]:
#verificar que la suma de las filas es uno
attn_weights.sum(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [28]:
#3. Calculas los vectores de contexto 
attn_context_vectors = attn_weights @ inputs
print(attn_context_vectors)
print(attn_weights.shape)
print(inputs.shape)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])
torch.Size([6, 6])
torch.Size([6, 3])



[Implemenatación de la autoatención con pesos entrenables](./4_autoatencion_pesos_entrenables.ipynb)